In [75]:
import dtale
import os
import pandas as pd
import numpy as np

In [10]:
print(os.getcwd())
dpath = os.path.join(os.getcwd(), "..\\space_titanic_data")
print(dpath)

C:\ALEX\Kaggle\space_titanic
C:\ALEX\Kaggle\space_titanic\..\space_titanic_data


In [12]:
files1 = os.listdir(dpath)
files2 = [os.path.join(dpath, f) for f in files1]
files = [f for f in files2 if os.path.isfile(f)]
print(files)

['C:\\ALEX\\Kaggle\\space_titanic\\..\\space_titanic_data\\sample_submission.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\space_titanic_data\\test.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\space_titanic_data\\train.csv']


In [29]:
df = pd.read_csv(files[2])  # the training data

In [119]:
df.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

## One-Hot-Encoder for HomePlanet

In [127]:
df['Home_Earth'] = df.apply(lambda row: 1 if row['HomePlanet'] == 'Earth' else 0, axis=1)
df['Home_Europa'] = df.apply(lambda row: 1 if row['HomePlanet'] == 'Europa' else 0, axis=1)
df['Home_Mars'] = df.apply(lambda row: 1 if row['HomePlanet'] == 'Mars' else 0, axis=1)

## One-Hot-Encoder for CryoSleep

In [129]:
df['cryo_sleep_true'] = df.apply(lambda row: 1 if row['CryoSleep'] == True else 0, axis=1)
df['cryo_sleep_false'] = df.apply(lambda row: 1 if row['CryoSleep'] == False else 0, axis=1)

## One-Hot-Encoder for Destination

In [132]:
df['destination_cancri'] = df.apply(lambda row: 1 if row['Destination'] == '55 Cancri e' else 0, axis=1)
df['destination_pso'] = df.apply(lambda row: 1 if row['Destination'] == 'PSO J318.5-22' else 0, axis=1)
df['destination_trappist'] = df.apply(lambda row: 1 if row['Destination'] == 'TRAPPIST-1e' else 0, axis=1)

## One-Hot-Encoder for VIP

In [133]:
df['vip_true'] = df.apply(lambda row: 1 if row['VIP'] == True else 0, axis=1)
df['vip_false'] = df.apply(lambda row: 1 if row['VIP'] == False else 0, axis=1)

## One-Hot-Encoder for Cabin

In [184]:
ccc = df['Cabin'].str.split("/").values
c = [k[0] for k in aaa if type(k) == list]
cc = list(set(c))
cc.sort()
print(cc)
cc_d = {cc[i]:(i+1) for i in range(len(cc))}
print(cc_d)
s = [k[2] for k in aaa if type(k) == list]
ss = list(set(s))
ss.sort()
print(ss)
ss_d = {ss[i]:(i+1) for i in range(len(ss))}
print(ss_d)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T']
{'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'T': 8}
['P', 'S']
{'P': 1, 'S': 2}


In [176]:
df['cabin_row'] = df.apply(lambda row: cc_d[row['Cabin'].split("/")[0]] if type(row['Cabin']) == list else 0, axis=1)
df['cabin_t'] = df.apply(lambda row: 1 if type(row['Cabin']) == list and row['Cabin'].split("/")[0] == 'T' else 0, axis=1)
df['cabin_num'] = df.apply(lambda row: int(row['Cabin'].split("/")[1]) if type(row['Cabin']) == list else 0, axis=1)
df['cabin_side'] = df.apply(lambda row: ss_d[row['Cabin'].split("/")[2]] if type(row['Cabin']) == list else 0, axis=1)

## Normalize Age

In [186]:
max_age = max(df['Age'].values)
df['age_norm'] = df.apply(lambda row: row['Age']/max_age if not pd.isnull(row['Age']) else 0, axis=1)

## Dropping the original columns (already translated to one-hot-encoder)

In [191]:
dfc = df.drop(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP'], axis=1)

In [193]:
dfc.columns

Index(['PassengerId', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa',
       'VRDeck', 'Name', 'Transported', 'Home_Earth', 'Home_Europa',
       'Home_Mars', 'cryo_sleep_true', 'cryo_sleep_false',
       'destination_cancri', 'destination_pso', 'destination_trappist',
       'vip_true', 'vip_false', 'cabin_t', 'cabin_row', 'cabin_num',
       'cabin_side', 'age_norm'],
      dtype='object')

## Assuming 0 billing for missing billings 

In [196]:
dfc['RoomService'].fillna(0, inplace=True)
dfc['FoodCourt'].fillna(0, inplace=True)
dfc['ShoppingMall'].fillna(0, inplace=True)
dfc['Spa'].fillna(0, inplace=True)
dfc['VRDeck'].fillna(0, inplace=True)

## Drop passanger names column

In [197]:
dfc = dfc.drop(['Name'], axis=1)

In [198]:
dfc.columns

Index(['PassengerId', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa',
       'VRDeck', 'Transported', 'Home_Earth', 'Home_Europa', 'Home_Mars',
       'cryo_sleep_true', 'cryo_sleep_false', 'destination_cancri',
       'destination_pso', 'destination_trappist', 'vip_true', 'vip_false',
       'cabin_t', 'cabin_row', 'cabin_num', 'cabin_side', 'age_norm'],
      dtype='object')

## Numeric types and No missing values remained

In [201]:
dfc.dtypes

PassengerId              object
RoomService             float64
FoodCourt               float64
ShoppingMall            float64
Spa                     float64
VRDeck                  float64
Transported                bool
Home_Earth                int64
Home_Europa               int64
Home_Mars                 int64
cryo_sleep_true           int64
cryo_sleep_false          int64
destination_cancri        int64
destination_pso           int64
destination_trappist      int64
vip_true                  int64
vip_false                 int64
cabin_t                   int64
cabin_row                 int64
cabin_num                 int64
cabin_side                int64
age_norm                float64
dtype: object

In [200]:
dfc[dfc.isnull().sum(axis=1) > 1]

,PassengerId,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Home_Earth,Home_Europa,Home_Mars,...,destination_cancri,destination_pso,destination_trappist,vip_true,vip_false,cabin_t,cabin_row,cabin_num,cabin_side,age_norm


## Feature Importance - method 1: XGBoost

In [44]:
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
from matplotlib import pyplot

In [50]:
X = df.loc[:, df.columns != 'Transported']
y = df.loc[:, df.columns == 'Transported']
model = XGBClassifier()
model.fit(X, y)
importance = model.feature_importances_
# pyplot.bar([x for x in range(len(importance))], importance)
# pyplot.show()

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:PassengerId: object, HomePlanet: object, CryoSleep: object, Cabin: object, Destination: object, VIP: object, Name: object

## Feature Importance - method 1: Permutation

In [39]:
dfnan = df[['HomePlanet', 'CryoSleep', 'Cabin', 'Age']]

In [40]:
drop_idx = []

In [42]:
dfnan[dfnan.isnull().sum(axis=1) > 1]

,PassengerId,HomePlanet,CryoSleep,Cabin,Age
314,0348_02,Mars,NaN,NaN,36.0
691,0715_01,Europa,NaN,NaN,54.0
1049,1116_02,Earth,NaN,NaN,36.0
1219,1295_01,Earth,NaN,NaN,19.0
1550,1645_01,NaN,True,NaN,18.0
1714,1823_01,NaN,False,NaN,67.0
2227,2384_07,NaN,True,NaN,11.0
2762,2970_01,Earth,False,NaN,NaN
3556,3821_01,Earth,False,NaN,NaN
3622,3896_01,NaN,NaN,C/140/S,18.0


In [37]:
drop_idx += list(dfnan[dfnan.isnull().sum(axis=1) > 2].index)

In [38]:
dfnan[dfnan.isnull().sum(axis=1) > 1]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age
314,0348_02,Mars,NaN,NaN,TRAPPIST-1e,36.0
347,0382_01,NaN,False,G/64/P,NaN,23.0
691,0715_01,Europa,NaN,NaN,55 Cancri e,54.0
1049,1116_02,Earth,NaN,NaN,TRAPPIST-1e,36.0
1203,1284_01,Mars,True,F/247/S,NaN,NaN
1219,1295_01,Earth,NaN,NaN,TRAPPIST-1e,19.0
1550,1645_01,NaN,True,NaN,55 Cancri e,18.0
1630,1727_01,Mars,True,F/342/P,NaN,NaN
1714,1823_01,NaN,False,NaN,TRAPPIST-1e,67.0
2227,2384_07,NaN,True,NaN,TRAPPIST-1e,11.0


In [24]:
df[df.isnull().sum(axis=1) > 2]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
1203,1284_01,Mars,True,F/247/S,NaN,NaN,False,0.0,NaN,0.0,0.0,0.0,Hal Knité,True
2639,2822_02,Earth,NaN,G/450/S,TRAPPIST-1e,5.0,NaN,0.0,0.0,0.0,0.0,NaN,Salley Harverez,False
2762,2970_01,Earth,False,NaN,TRAPPIST-1e,NaN,False,740.0,82.0,6.0,NaN,1.0,Dwin Adkinson,False
3072,3315_01,Earth,NaN,F/627/S,TRAPPIST-1e,15.0,False,10.0,0.0,99.0,NaN,2031.0,NaN,False
3535,3790_01,NaN,True,G/620/P,TRAPPIST-1e,13.0,False,0.0,NaN,0.0,NaN,0.0,Trick Meyersones,True
3882,4167_01,Earth,False,NaN,PSO J318.5-22,NaN,NaN,0.0,440.0,0.0,0.0,334.0,Ninaha Deckerson,False
4164,4446_05,Europa,NaN,B/175/S,TRAPPIST-1e,33.0,False,0.0,4017.0,NaN,NaN,2260.0,Phah Chocaters,True
4548,4840_01,NaN,True,F/915/S,TRAPPIST-1e,36.0,False,0.0,0.0,NaN,0.0,0.0,NaN,True
5208,5555_01,NaN,False,G/896/S,NaN,43.0,NaN,1.0,0.0,213.0,7.0,701.0,Winia Blanglison,True
5409,5777_01,Earth,NaN,F/1199/P,PSO J318.5-22,46.0,NaN,559.0,25.0,NaN,22.0,765.0,Katen River,False


In [30]:
dtale.show(df)

C:\Users\alext\anaconda3_64bit\lib\site-packages\dtale\views.py:755: FutureWarning:

['HomePlanet', 'Cabin', 'Destination', 'Name'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.



2023-02-12 22:58:24,649 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:24,672 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:24,680 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:24,687 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:24,693 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:24,701 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:24,707 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:24,714 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:24,721 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:24,727 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:24,735 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:24,742 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:24,751 - WA

2023-02-12 22:58:25,488 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:25,496 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:25,502 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:25,509 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:25,517 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:25,524 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:25,533 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:25,540 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:25,548 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:36,869 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:36,878 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:36,885 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:36,892 - WA

2023-02-12 22:58:47,856 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:47,865 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:47,872 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:47,880 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:47,887 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:47,894 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:47,901 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:47,907 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:47,915 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:47,922 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:47,929 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:47,936 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-02-12 22:58:47,943 - WA